In [3]:
import re
import os
import numpy as np
import pandas as pd
# import jieba
# import jieba.analyse  
import matplotlib.pyplot as plt
from tqdm import tqdm

# import gensim
# from gensim.models import word2vec
# from gensim.models import KeyedVectors

In [4]:
def load_data():
    df_tf_idf = pd.read_csv('df_tf_idf.csv',index_col=False).drop(columns='Unnamed: 0')
#     df_tf_idf = pd.read_csv('df_tf_idf.csv',index_col=False)
    print(df_tf_idf.head())
    
    ori_text = pd.read_csv('original_text.csv')
    print(ori_text.head(2))
    
    return df_tf_idf,ori_text

In [5]:
df_tf_idf,ori_text = load_data()

  Keyword    TF_IDF
0      北京  0.650746
1      暴雨  0.488569
2      大雨  0.326697
3      积水  0.125624
4     遇难者  0.068973
                                            new_text
0  这是不是无耻到一定境界了抗击暴雨的普通百姓居然是北京精神的作用人们的团结热心居然是某几位领导...
1                                北京市民坚持住众志成城定可度过暴雨阶段


In [33]:
keyword = df_tf_idf['Keyword'].tolist()
word_num = len(keyword)
M = [[0 for i in range(word_num)] for j in range(word_num)]
doc_dict = dict()

In [16]:
for i in tqdm(range(len(ori_text['new_text']))):
    text = ori_text['new_text'][i]
    doc_id = i
    for w in range(len(keyword)):
        word = keyword[w]
#         print(word)
        if word in text:
            if word in doc_dict:
                doc_dict[word].append(i)
            else:
                doc_dict[word] = [i]
            for k in range(len(keyword)):
                other_word = keyword[k]
                if (other_word != word) and (other_word in text):
                    M[w][k] += 1

100%|██████████| 41642/41642 [00:07<00:00, 5321.43it/s]


In [55]:
df_M = pd.DataFrame(M, columns=keyword, index=keyword)

In [57]:
df_M.to_csv('共现矩阵.csv')

In [37]:
df_M = pd.read_csv('共现矩阵.csv').drop(columns = ['Unnamed: 0'])

In [40]:
M = df_M.values

In [41]:
Matrix_lst = list()
for w in tqdm(range(len(keyword))):
    for k in range(w,len(keyword)):
        if M[w][k] != 0:
            Matrix_lst.append([keyword[w],keyword[k],'Undirected',M[w][k]])

100%|██████████| 100/100 [00:00<00:00, 440.70it/s]


In [42]:
df_matrix_lst = pd.DataFrame(Matrix_lst, columns = ['Source','Target','Type','Weight'])
df_matrix_lst.head()

,Source,Target,Type,Weight
0,北京,暴雨,Undirected,39846
1,北京,大雨,Undirected,26818
2,北京,积水,Undirected,9232
3,北京,遇难者,Undirected,4426
4,北京,悼念,Undirected,3950


In [44]:
df_matrix_lst.shape

(4289, 4)

In [45]:
df_matrix_lst.to_csv('共现矩阵_列表.csv',index=False)

# 共现矩阵——排序筛选

In [46]:
df_matrix = pd.read_csv('共现矩阵_列表.csv')
df_matrix.shape

(4289, 4)

In [47]:
df_matrix_sorted = df_matrix.sort_values('Weight', ascending = False)

In [59]:
df_matrix_sorted_40 = df_matrix_sorted[df_matrix_sorted['Weight']>40]
df_matrix_sorted_40 = df_matrix_sorted_40[df_matrix_sorted_40['Weight']<5000]
df_matrix_sorted_40.shape

(1549, 4)

In [61]:
df_matrix_sorted_40.to_csv('共现矩阵_列表_40.csv', index=False)

# 生成节点信息

In [12]:
temp = pd.read_csv('共现矩阵_列表_40.csv')
word_set = set(temp['Source'])

In [21]:
new_df = []
for d in df_tf_idf.values.tolist():
    if(d[0] in word_set):
        newd = [d[0], d[0],d[1]]
        new_df.append(newd)

In [22]:
new_df_1 = pd.DataFrame(new_df, columns = ['Id','Label','size'])
new_df_1.to_csv('共现矩阵_节点_40.csv',index=False, encoding="UTF-8")